In [1]:
import requests
import json
import pandas as pd
import altair as alt

/Users/matthewyau/opt/anaconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# Plan

In [2]:
#GRAPHS WE MIGHT WANT TO IMPLEMENT:

#Gaming preferences of friend group
    #Get common games (value counts of games that most people own)
    #Favorite games (show top 3 most played game)
    #Compare genres between two friends


#Leaderboards
    #Total number of hours played
    #Biggest spender
    #Biggest collector (greatest number of games owned)
    #Biggest collector of unplayed games (greatest number of games owned that do not have any hours played in it)



In [3]:
#How to plot images of people
source = pd.DataFrame.from_records([
      {"x": 0.5, "y": 0.5, "img": "https://avatars.akamai.steamstatic.com/9f5ee8f8c19201896d678324160c777d25ff157d.jpg"}


])

alt.Chart(source).mark_image(
    width=50,
    height=50
).encode(
    x='x',
    y='y',
    url='img'
)



alt.Chart(...)

# Code

#Always enter the SteamID I pasted below to search for people

In [4]:
#SteamId = 76561197996661065

In [5]:
###### We don't necessarily need this function, I built it so we can retrieve Steam ID by searching a profile name, but 
###### the problem is too many profile share same name so can't retrieve steam ID of what we want

###### Delete this function and then plug in steam ID directly into second code block

#Implementation:
#Search for steam ID using a person's name

theInput = str(input("Please enter a name or ID you would like to search the game library of"))

try:
    steamId = int(theInput)
except:
    searchQuery = ""
    if len(theInput.split()) != 1:
        for word in theInput.split():
            searchQuery += word + "&nbsp"
    steamId = int(requests.get("http://api.steampowered.com/ISteamUser/ResolveVanityURL/v0001/?key=3D41F12368AF3E305A8233ABFB965CA2&vanityurl=" + searchQuery).json()["response"]["steamid"])


    
    


Please enter a name or ID you would like to search the game library of 76561197996661065


#### Get a person's friend list off steam ID 

In [6]:
#Implementation:
#Get friend list based off of Steam ID
friendlist = requests.get("http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key=3D41F12368AF3E305A8233ABFB965CA2&relationship=friend&steamid=" + str(steamId)).json()
friendIds = []
friendIds.append(str(steamId))

for friend in friendlist["friendslist"]["friends"]:
    friendIds.append(friend["steamid"])

#### Get all of your own and friends names and profile pictures 

In [7]:
friendSearchQuery = ""
for i in friendIds:
    friendSearchQuery += i + ","
    
#Implementation: 
#Get friend name:
friendName = requests.get("http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=3D41F12368AF3E305A8233ABFB965CA2&steamids=" + friendSearchQuery[:-1]).json()
friendNameList = {}

for friendName in friendName["response"]["players"]:
    friendNameList[friendName["personaname"]] = []
    friendNameList[friendName["personaname"]].append(friendName["steamid"])
    friendNameList[friendName["personaname"]].append(friendName["avatar"])
    

#friendNameList

#### Get all of your own and friends game libraries

In [8]:
#Implementation:
#Append game library
for friend in friendNameList:
    gameLibrary = requests.get("https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3D41F12368AF3E305A8233ABFB965CA2&include_appinfo=1&format=json&steamid="
                               + friendNameList[friend][0]).json()["response"]["games"]
    friendNameList[friend].append(gameLibrary)



#### Get most of game prices for games (if it exists)

In [9]:
#Implementation:
#Get game prices
appIdList = []

for friend in friendNameList:
    for game in friendNameList[friend][2]:
        appIdList.append(game["appid"])
        
appString = ""
for app in appIdList:
    appString = appString + str(app) + ","
prices = requests.get("https://store.steampowered.com/api/appdetails?appids=" + appString + "&filters=price_overview").json()    

for friend in friendNameList:
    for game in friendNameList[friend][2]:
        if prices[str(game["appid"])]["success"]:
            if prices[str(game["appid"])]["data"] != []:
                game["price"] = float(prices[str(game["appid"])]["data"]["price_overview"]["final_formatted"][5:])

#### Compare what genres you and your friend plays

In [10]:
#### This is an extra function that I think will be too much work to implement
#### Basically adds genres of games to game list, but we need to query each game individually
#### Because every friend has 40+ games, results in spamming Steam API too much and get IP banned
#### So can only compare two people's steam genre libraries: might be too much, can delete or implement if we have more time

#Implementation:
#Append game genre to game library.


def compare_Genres(personOne, personTwo):
    if (personOne in list(friendNameList.keys())) & (personTwo in list(friendNameList.keys())):
        i = 0
        filterName = [personOne, personTwo]
        filteredFriendNameList = dict((name, friendNameList[name]) for name in filterName if name in friendNameList)
    else:
        print("Name does not exist in the dictionary")
        return

    for friend in filteredFriendNameList:
        gameLibrary = filteredFriendNameList[friend][2]
        if i < 190:
            for game in gameLibrary:
                if game["playtime_forever"] > 100:
                    try:
                        appid = str(game["appid"])
                        gameGenre = requests.get("https://store.steampowered.com/api/appdetails/?appids=" + appid).json()[appid]["data"]["genres"]
                        game["genre"] = gameGenre
                        i += 1
                    except:
                        #print("Couldn't scrape", game)
                        pass
                else:
                    pass
        else:
            print("Cannot exceed more than 200 requests in 5 minutes")
    return filteredFriendNameList


In [11]:
#theList = compare_Genres("Waul Palker", "HenryL")


In [12]:
#theList

#### Code that converts the extracted results into a dataframe

In [13]:


#Implementation:
#Convert friendList to pandas dataframe
fullLst = []

for friend in friendNameList:
    lst = []
    lst.append(friend) #Append friend name to list
    lst.append(friendNameList[friend][1]) #Append friend avatar to list

    
    gameLst = [] #Start appending library of games to list
    for game in friendNameList[friend][2]: 
        gameLst.append(game["name"])   
    lst.append(gameLst) #End appending library of games to list
    
    totalNumGamesOwned = len(gameLst) #Append total number of games owned
    lst.append(totalNumGamesOwned)
    
    priceLst = []
    for game in friendNameList[friend][2]:
        try:
            priceLst.append(game["price"])
        except:
            pass
    lst.append(priceLst) #Append purchase price (full price, not on sale) for all games owned
    #lst.append(sum(priceLst)) #Uncomment this line if we want the sum of prices for all games owned instead of individual items
    
    playtimeLst = []
    for game in friendNameList[friend][2]:
        playtimeLst.append(game["playtime_forever"])
    lst.append(playtimeLst) #Appending total playtime for all games owned
    #lst.append(sum(playtimeLst))
    
    favoriteGames = pd.DataFrame(friendNameList[friend][2]).sort_values(by="playtime_forever", ascending = False)["name"].iloc[:3,].tolist() #Append top 3 games 
    lst.append(favoriteGames)
    
    gameImageLst = []
    for game in friendNameList[friend][2]:
        imageLink = "http://media.steampowered.com/steamcommunity/public/images/apps/"
        imageLink = imageLink + str(game["appid"]) + "/" + game["img_icon_url"] + ".jpg"
        gameImageLst.append(imageLink)
    
    lst.append(gameImageLst)
        
    

                                 
    
    fullLst.append(lst)
    


In [14]:
pd.DataFrame(fullLst, columns = ["Name", "ProfilePicture", "GameList", "TotalNumGames", "GamePrice", "Playtime", "FavoriteGames", "ImageLinks"])

,Name,ProfilePicture,GameList,TotalNumGames,GamePrice,Playtime,FavoriteGames,ImageLinks
0,HenryL,https://avatars.akamai.steamstatic.com/08666eb...,"[Total War: EMPIRE - Definitive Edition, Heart...",133,"[29.99, 2.74, 12.99, 10.99, 21.99, 35.99, 16.9...","[1777, 47, 258, 116, 2233, 624, 1942, 6307, 63...","[Mount & Blade: Warband, War Thunder, Stellaris]",[http://media.steampowered.com/steamcommunity/...
1,卷毛毛秀丽,https://avatars.akamai.steamstatic.com/2f66582...,"[The Forest, Papers, Please, Cuphead, Dark Dec...",30,"[22.79, 10.99, 21.99, 19.49, 29.99, 12.99, 22....","[491, 191, 4, 246, 0, 0, 0, 54, 0, 0, 246, 184...","[Sniper Ghost Warrior Contracts 2, The Forest,...",[http://media.steampowered.com/steamcommunity/...
2,马铃鼠鼠,https://avatars.akamai.steamstatic.com/302685d...,"[Counter-Strike, Counter-Strike: Condition Zer...",63,"[11.49, 11.49, 11.49, 12.99, 5.69, 5.69, 5.69,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Counter-Strike, Thronebreaker: The Witcher Ta...",[http://media.steampowered.com/steamcommunity/...
3,matt,https://avatars.akamai.steamstatic.com/4e8b518...,"[Counter-Strike: Source, Total War: MEDIEVAL I...",58,"[12.99, 29.99, 21.99, 29.99, 9.99, 35.99, 12.9...","[235, 10468, 255, 2010, 0, 2302, 594, 160, 464...","[Total War: MEDIEVAL II - Definitive Edition, ...",[http://media.steampowered.com/steamcommunity/...
4,audz,https://avatars.akamai.steamstatic.com/fef49e7...,"[Portal 2, Terraria, Starbound, Starbound - Un...",19,"[12.99, 10.99, 16.99, 27.99, 16.99, 33.99, 77....","[2897, 56, 3039, 0, 88, 11936, 1801, 128, 120,...","[Stardew Valley, Spiritfarer®: Farewell Editio...",[http://media.steampowered.com/steamcommunity/...
5,Waul Palker,https://avatars.akamai.steamstatic.com/b0d0557...,"[Battlefield: Bad Company™ 2, Super Monday Nig...",40,"[26.99, 25.99, 59.99, 33.99, 5.69, 49.99, 45.9...","[0, 0, 0, 0, 0, 19, 0, 0, 1052, 0, 0, 0, 188, ...","[Counter-Strike: Global Offensive, PUBG: BATTL...",[http://media.steampowered.com/steamcommunity/...
6,Stealth,https://avatars.akamai.steamstatic.com/6a054c8...,"[Garry's Mod, Mass Effect (2007), Star Wars: B...",76,"[10.99, 26.99, 11.49, 11.49, 6.69, 6.69, 6.69,...","[29, 409, 18, 0, 0, 0, 0, 0, 44, 92, 966, 0, 0...","[The Elder Scrolls V: Skyrim, Fallout: New Veg...",[http://media.steampowered.com/steamcommunity/...
7,All About That Money,https://avatars.akamai.steamstatic.com/9f5ee8f...,"[Counter-Strike: Source, Midnight Club II, Bur...",49,"[12.99, 11.49, 11.49, 6.69, 6.69, 6.69, 11.49,...","[1063, 0, 0, 14, 0, 0, 0, 0, 0, 1, 431, 202, 2...","[Counter-Strike: Global Offensive, Counter-Str...",[http://media.steampowered.com/steamcommunity/...
8,Tedd2008,https://avatars.akamai.steamstatic.com/f3065ac...,"[Terraria, Grand Theft Auto V, Mini Metro, AST...",23,"[10.99, 10.99, 34.99, 66.49, 21.99, 15.49, 39....","[511, 12841, 17, 654, 585, 436, 1012, 1797, 15...","[NBA 2K22, F1 2021, Grand Theft Auto V]",[http://media.steampowered.com/steamcommunity/...


### SCRAP CODE

In [52]:


for friend in friendNameList:
    lst = []
    lst.append(friend)
    lst.append(friendNameList[friend][1]) #Append friend name to list
    lst.append(friendNameList[friend][2]) #Append friend avatar to list
    
    gameLst = []
    for game in friendNameList[friend][2]:
        gameLst.append(game["name"])
    fullLst.append(lst)
#fullLst


#[sum(priceGameList) for priceGameList in sampleData[4]]
#for friend in friendNameList:
#    for game in friendNameList[friend][2]:
#        print(game["name"])

pd.DataFrame(friendNameList["Stealth"][2])["playtime_forever"].sum()

pd.DataFrame(["Stealth", 200], ["Hi", 200])
pd.DataFrame(friendNameList["HenryL"][2])



#for friend in theList:
#    for game in theList[friend][2]:
#        try:
#            print(game["genre"])
#        except:
#            pass

#for friend in filteredFriendNameList:
#    gameLibrary = filteredFriendNameList[friend][2]
#    for game in gameLibrary:
#        if game["playtime_forever"] > 100:
#            print(game)


#filteredFriendNameList

amountList = []

for friend in friendNameList:
    gameLibrary = friendNameList[friend][2]
    for game in gameLibrary:
        if game["playtime_forever"] > 100:
                amountList.append(game)


requests.get("https://store.steampowered.com/api/appdetails/?appids=242760").json()["242760"]["data"]["genres"]

friendNameList["马铃鼠鼠"][2][0]["genre"] = requests.get("https://store.steampowered.com/api/appdetails/?appids=242760").json()["242760"]["data"]["genres"]

import pandas as pd
for i in pd.DataFrame(friendNameList["Stealth"][2])["price"]:
    print(i["data"]["price_overview"]["fi)

#How to plot images of games
#https://stackoverflow.com/questions/53963328/how-do-i-get-a-hash-for-a-picture-form-a-steam-game

"http://media.steampowered.com/steamcommunity/public/images/apps/17450/8cc583d72b84ae9a4026c762e130869127dd05e4.jpg"

#After apps, id
#After id, put link
#Add jpg to the end
                                      
                                      
                                      allImageLinks = []
for friend in friendNameList:
    for game in friendNameList[friend][2]:
        imageLink = "http://media.steampowered.com/steamcommunity/public/images/apps/"
        if str(game["img_icon_url"]) != "":
            imageLink = imageLink + str(game["appid"]) + "/" + str(game["img_icon_url"]) + ".jpg"
        else:
            allImageLinks.append("")
        allImageLinks.append(imageLink)
                                      
#for friend in friendNameList:
#    for game in friendNameList[friend][2]:
#        print(game["img_icon_url"])
#
                                      
#friendNameList["HenryL"][2]
                                      
                                      
#friendNameList
                                      
                                      
#requests.get("https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3D41F12368AF3E305A8233ABFB965CA2&include_appinfo=1&format=json&steamid=76561198417923785").json()                                      